In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('train.csv')
df.shape

(404290, 6)

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [9]:
df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,1
question2,2
is_duplicate,0


In [7]:
df.duplicated().sum()

0

In [8]:
# distribution of duplicate an non duplicate
print(df.duplicated().value_counts())

False    404290
Name: count, dtype: int64


In [27]:
df=df.sample(5000)

In [11]:
df.shape

(5000, 6)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 325390 to 149609
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            5000 non-null   int64 
 1   qid1          5000 non-null   int64 
 2   qid2          5000 non-null   int64 
 3   question1     5000 non-null   object
 4   question2     5000 non-null   object
 5   is_duplicate  5000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 273.4+ KB


In [13]:
df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [14]:
que_df= df[['question1','question2']]
que_df.head()

,question1,question2
325390,Would post-soviet Russia chose Vladivostok as ...,How can I relieve collar bone pain?
151483,What is the most compelling human encounter wi...,What is the most compelling alien encounter st...
30601,"What is your most favorite picture of you, and...","What are your favorite pictures, and why?"
401637,How can we image subterranean very small and b...,Can intense and coherent extremely-low-frequen...
16703,To what extent has Gibbon's History of the Dec...,"Is it worthwhile, in this day and age, to read..."


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
# merge text
questions = list(que_df['question1']) + list(que_df['question2'])
cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [46]:
temp_df1 = pd.DataFrame(q1_arr, index= que_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= que_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(5000, 6000)

In [47]:
temp_df1['is_duplicate'] = df['is_duplicate']

In [48]:
temp_df1.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
325390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
151483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
30601,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,NaN
401637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
16703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df1.iloc[:,0:-1], temp_df1.iloc[:,-1], test_size=0.33, random_state=42)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7018181818181818

#Feature engineering

In [30]:
df['q1_len'] = df['question1'].str.len()
df['q2_len'] = df['question2'].str.len()

In [31]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len
40022,40022,72477,72478,How do I find the correct Wells Fargo routing ...,How can I find my Wells Fargo account number o...,0,73,83
378220,378220,509604,509605,Can a pentagon tessellate? Why or why not?,Does a pentagon tessellate? Why or why not? /,1,42,45
295117,295117,99864,43379,How is the word 'susceptible' used in a sentence?,How is the word 'quibble' used in a sentence?,0,49,45
317202,317202,109191,52602,"What is the property tax rate in Granville, Oh...","What is the property tax rate in Granville, Oh...",0,94,91
45306,45306,81210,81211,Are Star Wars and Star Trek related?,Which is better and why: Star Wars or Star Trek?,0,36,48


In [32]:
df['q1_num_words'] = df['question1'].apply(lambda row: len(row.split(" ")))
df['q2_num_words'] = df['question2'].apply(lambda row: len(row.split(" ")))
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words
40022,40022,72477,72478,How do I find the correct Wells Fargo routing ...,How can I find my Wells Fargo account number o...,0,73,83,14,14
378220,378220,509604,509605,Can a pentagon tessellate? Why or why not?,Does a pentagon tessellate? Why or why not? /,1,42,45,8,9
295117,295117,99864,43379,How is the word 'susceptible' used in a sentence?,How is the word 'quibble' used in a sentence?,0,49,45,9,9
317202,317202,109191,52602,"What is the property tax rate in Granville, Oh...","What is the property tax rate in Granville, Oh...",0,94,91,18,18
45306,45306,81210,81211,Are Star Wars and Star Trek related?,Which is better and why: Star Wars or Star Trek?,0,36,48,7,10


In [33]:
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return len(w1 & w2)

In [34]:
df['word_common'] = df.apply(common_words, axis=1)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common
40022,40022,72477,72478,How do I find the correct Wells Fargo routing ...,How can I find my Wells Fargo account number o...,0,73,83,14,14,7
378220,378220,509604,509605,Can a pentagon tessellate? Why or why not?,Does a pentagon tessellate? Why or why not? /,1,42,45,8,9,6
295117,295117,99864,43379,How is the word 'susceptible' used in a sentence?,How is the word 'quibble' used in a sentence?,0,49,45,9,9,8
317202,317202,109191,52602,"What is the property tax rate in Granville, Oh...","What is the property tax rate in Granville, Oh...",0,94,91,18,18,15
45306,45306,81210,81211,Are Star Wars and Star Trek related?,Which is better and why: Star Wars or Star Trek?,0,36,48,7,10,3


In [35]:
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return (len(w1) + len(w2))

In [36]:
df['word_total'] = df.apply(total_words, axis=1)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total
40022,40022,72477,72478,How do I find the correct Wells Fargo routing ...,How can I find my Wells Fargo account number o...,0,73,83,14,14,7,28
378220,378220,509604,509605,Can a pentagon tessellate? Why or why not?,Does a pentagon tessellate? Why or why not? /,1,42,45,8,9,6,15
295117,295117,99864,43379,How is the word 'susceptible' used in a sentence?,How is the word 'quibble' used in a sentence?,0,49,45,9,9,8,18
317202,317202,109191,52602,"What is the property tax rate in Granville, Oh...","What is the property tax rate in Granville, Oh...",0,94,91,18,18,15,32
45306,45306,81210,81211,Are Star Wars and Star Trek related?,Which is better and why: Star Wars or Star Trek?,0,36,48,7,10,3,15


In [37]:
df['word_share'] = round(df['word_common']/df['word_total'],2)
df.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
40022,40022,72477,72478,How do I find the correct Wells Fargo routing ...,How can I find my Wells Fargo account number o...,0,73,83,14,14,7,28,0.25
378220,378220,509604,509605,Can a pentagon tessellate? Why or why not?,Does a pentagon tessellate? Why or why not? /,1,42,45,8,9,6,15,0.40


In [38]:
que_df.head(2)

,question1,question2
325390,Would post-soviet Russia chose Vladivostok as ...,How can I relieve collar bone pain?
151483,What is the most compelling human encounter wi...,What is the most compelling alien encounter st...


In [44]:
final_df=df.drop(columns=['id','qid1','qid2','question1','question2'])
final_df.head()

,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
40022,0,73,83,14,14,7,28,0.25
378220,1,42,45,8,9,6,15,0.40
295117,0,49,45,9,9,8,18,0.44
317202,0,94,91,18,18,15,32,0.47
45306,0,36,48,7,10,3,15,0.20


In [50]:
temp_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
325390,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
151483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
final_df=pd.concat([final_df,temp_df],axis=1)
final_df.head()

,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,0,1,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
40022,0.0,73.0,83.0,14.0,14.0,7.0,28.0,0.25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378220,1.0,42.0,45.0,8.0,9.0,6.0,15.0,0.40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295117,0.0,49.0,45.0,9.0,9.0,8.0,18.0,0.44,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317202,0.0,94.0,91.0,18.0,18.0,15.0,32.0,0.47,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45306,0.0,36.0,48.0,7.0,10.0,3.0,15.0,0.20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# replacing NaN with 0
final_df.fillna(0,inplace=True)
final_df.head()

,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,0,1,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
40022,0.0,73.0,83.0,14.0,14.0,7.0,28.0,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378220,1.0,42.0,45.0,8.0,9.0,6.0,15.0,0.40,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295117,0.0,49.0,45.0,9.0,9.0,8.0,18.0,0.44,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317202,0.0,94.0,91.0,18.0,18.0,15.0,32.0,0.47,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45306,0.0,36.0,48.0,7.0,10.0,3.0,15.0,0.20,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
final_df.isnull().sum()

,0
is_duplicate,0
q1_len,0
q2_len,0
q1_num_words,0
q2_num_words,0
...,...
2995,0
2996,0
2997,0
2998,0


In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=42)
#

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8333333333333334